These packages allow us to reshape and analyze the data

In [1]:
import pandas as pd
import numpy as np

This data comes from here: https://www.phoenixopendata.com/dataset/crime-data

In [2]:
zip_crimes = pd.read_csv('crimestat.csv')

C:\Users\Sterling\AnacondaNew\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


This data comes from https://data.census.gov using 5-digit ZIP codes in Arizona and the 2021 American Community Survey as filters. I selected tables S0101 and S1401.

In [3]:
zip_tot_pop = pd.read_csv('ACSST5Y2021.S0101-Data.csv')

In [82]:
zip_enrolled_school = pd.read_csv('ACSST5Y2021.S1401-Data.csv')

This section uses Pandas to group the crime data by ZIP code to get a total count of crimes per ZIP code over the period

In [15]:
zip_crimes_tot = zip_crimes.groupby('ZIP')['UCR CRIME CATEGORY'].count().reset_index()
zip_crimes_tot.columns = ['ZIP','crime_count']

In [16]:
zip_crimes_tot.head()

,ZIP,crime_count
0,85003.0,6093
1,85004.0,6281
2,85006.0,9298
3,85007.0,8547
4,85008.0,22483


This section uses census data to get the 2021 population for each ZIP code in the Phoenix area

In [17]:
zip_tot_pop.columns

Index(['GEO_ID', 'NAME', 'S0101_C01_001E', 'S0101_C01_001M', 'S0101_C01_001MA',
       'S0101_C01_001EA', 'S0101_C01_002E', 'S0101_C01_002M',
       'S0101_C01_002MA', 'S0101_C01_002EA',
       ...
       'S0101_C06_036MA', 'S0101_C06_037E', 'S0101_C06_037EA',
       'S0101_C06_037M', 'S0101_C06_037MA', 'S0101_C06_038E',
       'S0101_C06_038EA', 'S0101_C06_038M', 'S0101_C06_038MA', 'Unnamed: 914'],
      dtype='object', length=915)

In [18]:
zip_tot_pop_slice = zip_tot_pop[['NAME', 'S0101_C01_001E']]

In [19]:
zip_tot_pop_slice = zip_tot_pop_slice.copy(deep=True)

This section extracts the ZIP code from the NAME field of the census data and converts it (and the population) into a number

In [25]:
def get_zip(cell):
    try:
        cell_split = cell.split(' ')
        return cell_split[-1]
    except:
        return cell

In [26]:
zip_tot_pop_slice['NAME'] = zip_tot_pop_slice['NAME'].apply(get_zip)

In [28]:
zip_tot_pop_slice.columns = ['ZIP','tot_pop']

In [34]:
zip_tot_pop_slice['ZIP'] = pd.to_numeric(zip_tot_pop_slice['ZIP'], errors='coerce')
zip_tot_pop_slice['tot_pop'] = pd.to_numeric(zip_tot_pop_slice['tot_pop'], errors='coerce')

In [35]:
zip_tot_pop_slice.head()

,ZIP,tot_pop
0,NaN,NaN
1,84531.0,478.0
2,84536.0,1495.0
3,85003.0,8105.0
4,85004.0,9071.0


In this section we merge the crime data and the population data using the common key of ZIP code

In [39]:
zip_crime_rate = zip_crimes_tot.merge(zip_tot_pop_slice)

This section calculates a crime rate for each ZIP code by dividing its crime count by its population figure

In [41]:
zip_crime_rate['crime_rate'] = zip_crime_rate['crime_count'] / zip_crime_rate['tot_pop']

In [42]:
zip_crime_rate.head()

,ZIP,crime_count,tot_pop,crime_rate
0,85003.0,6093,8105.0,0.751758
1,85003.0,6093,8105.0,0.751758
2,85004.0,6281,9071.0,0.692426
3,85004.0,6281,9071.0,0.692426
4,85006.0,9298,24777.0,0.375267


This section gets the percentage enrolled for various age groups by ZIP code and converts it to a number in decimal format

In [84]:
zip_enrolled_school.columns

Index(['GEO_ID', 'NAME', 'S1401_C01_001E', 'S1401_C01_001EA', 'S1401_C01_001M',
       'S1401_C01_001MA', 'S1401_C01_002E', 'S1401_C01_002M',
       'S1401_C01_002MA', 'S1401_C01_002EA',
       ...
       'S1401_C06_032MA', 'S1401_C06_033E', 'S1401_C06_033EA',
       'S1401_C06_033M', 'S1401_C06_033MA', 'S1401_C06_034E', 'S1401_C06_034M',
       'S1401_C06_034MA', 'S1401_C06_034EA', 'Unnamed: 818'],
      dtype='object', length=819)

In [86]:
zip_enrolled_school_slice = zip_enrolled_school[['NAME','S1401_C02_020E','S1401_C02_022E','S1401_C02_024E','S1401_C02_026E']]

In [87]:
zip_enrolled_school_slice = zip_enrolled_school_slice.copy(deep=True)

In [88]:
zip_enrolled_school_slice['NAME'] = zip_enrolled_school_slice['NAME'].apply(get_zip)

In [89]:
zip_enrolled_school_slice.columns = ['ZIP','percent_15_17_enrolled_school','percent_18_19_enrolled_school',
                                           'percent_20_24_enrolled_school','percent_25_34_enrolled_school']

In [90]:
zip_enrolled_school_slice['ZIP'] = pd.to_numeric(zip_enrolled_school_slice['ZIP'], errors='coerce')
zip_enrolled_school_slice['percent_15_17_enrolled_school'] = pd.to_numeric(
    zip_enrolled_school_slice['percent_15_17_enrolled_school'], errors='coerce')
zip_enrolled_school_slice['percent_18_19_enrolled_school'] = pd.to_numeric(
    zip_enrolled_school_slice['percent_18_19_enrolled_school'], errors='coerce')
zip_enrolled_school_slice['percent_20_24_enrolled_school'] = pd.to_numeric(
    zip_enrolled_school_slice['percent_20_24_enrolled_school'], errors='coerce')
zip_enrolled_school_slice['percent_25_34_enrolled_school'] = pd.to_numeric(
    zip_enrolled_school_slice['percent_25_34_enrolled_school'], errors='coerce')

In [91]:
zip_enrolled_school_slice['percent_15_17_enrolled_school'] = zip_enrolled_school_slice['percent_15_17_enrolled_school'] / 100
zip_enrolled_school_slice['percent_18_19_enrolled_school'] = zip_enrolled_school_slice['percent_18_19_enrolled_school'] / 100
zip_enrolled_school_slice['percent_20_24_enrolled_school'] = zip_enrolled_school_slice['percent_20_24_enrolled_school'] / 100
zip_enrolled_school_slice['percent_25_34_enrolled_school'] = zip_enrolled_school_slice['percent_25_34_enrolled_school'] / 100

In [92]:
zip_enrolled_school_slice.head()

,ZIP,percent_15_17_enrolled_school,percent_18_19_enrolled_school,percent_20_24_enrolled_school,percent_25_34_enrolled_school
0,NaN,NaN,NaN,NaN,NaN
1,NaN,0.964,0.745,0.409,0.114
2,84531.0,0.333,0.357,0.000,0.435
3,84536.0,1.000,0.645,0.290,0.091
4,85003.0,0.938,0.395,0.418,0.118


This section merges the crime rate dataframe with the percentage enrolled for various age groups, using the common key of ZIP code

In [93]:
zip_crime_school_rate = zip_crime_rate.merge(zip_enrolled_school_slice)

In [94]:
zip_crime_school_rate.head()

,ZIP,crime_count,tot_pop,crime_rate,percent_15_17_enrolled_school,percent_18_19_enrolled_school,percent_20_24_enrolled_school,percent_25_34_enrolled_school
0,85003.0,6093,8105.0,0.751758,0.938,0.395,0.418,0.118
1,85003.0,6093,8105.0,0.751758,0.938,0.395,0.418,0.118
2,85003.0,6093,8105.0,0.751758,0.938,0.395,0.418,0.118
3,85003.0,6093,8105.0,0.751758,0.938,0.395,0.418,0.118
4,85004.0,6281,9071.0,0.692426,0.738,0.980,0.541,0.118


This section downloads the above table as a csv file

In [97]:
zip_crime_school_rate.to_csv('zip_crime_school_rate.csv',index=False)

This section calculates the correlation, across the ZIP codes, between the crime rate and percent enrolled for various age groups. It finds that we could lower crime if we focused on helping youth between the ages of 15 and 17 as well as 20 to 24 to enroll and stay in school.

In [95]:
zip_crime_school_rate['crime_rate'].corr(zip_crime_school_rate['percent_15_17_enrolled_school'])

-0.1777534133297583

In [79]:
zip_crime_school_rate['crime_rate'].corr(zip_crime_school_rate['percent_18_19_enrolled_school'])

-0.0688509087861424

In [80]:
zip_crime_school_rate['crime_rate'].corr(zip_crime_school_rate['percent_20_24_enrolled_school'])

-0.18379422804721066

In [81]:
zip_crime_school_rate['crime_rate'].corr(zip_crime_school_rate['percent_25_34_enrolled_school'])

-0.08945473152755529